In [1]:
from spektral.datasets import citation
data = citation.load_data('cora')
A, X, y, train_mask, val_mask, test_mask = data

N = A.shape[0]
F = X.shape[-1]
n_classes = y.shape[-1]

Using TensorFlow backend.
/Users/em/anaconda3/envs/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


Loading cora dataset
Pre-processing node features


In [2]:
from spektral.layers import GraphConv
from keras.models import Model
from keras.layers import Input, Dropout

In [3]:
# Model definition
X_in = Input(shape=(F, ))  # Input layer for X
A_in = Input((N, ), sparse=True)  # Input layer for A

graph_conv_1 = GraphConv(16, activation='relu')([X_in, A_in])
dropout = Dropout(0.5)(graph_conv_1)
graph_conv_2 = GraphConv(n_classes, activation='softmax')([dropout, A_in])

# Build model
model = Model(inputs=[X_in, A_in], outputs=graph_conv_2)

W0909 09:57:18.131289 4398224832 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0909 09:57:18.142784 4398224832 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0909 09:57:18.144531 4398224832 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:539: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.

W0909 09:57:18.148360 4398224832 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.ran

In [4]:
from spektral import utils
A = utils.localpooling_filter(A)

In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

W0909 09:57:31.849871 4398224832 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2708)         0                                            
__________________________________________________________________________________________________
graph_conv_1 (GraphConv)        (None, 16)           22944       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 16)           0           graph_conv_1[0][0]         

In [6]:
# Train model
validation_data = ([X, A], y, val_mask)
model.fit([X, A],
          y,
          sample_weight=train_mask,
          epochs=100,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False)  # Shuffling data means shuffling the whole graph

W0909 09:57:39.265661 4398224832 deprecation.py:323] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2708 samples, validate on 2708 samples
Epoch 1/100
2708/2708 [==============================] - 0s 120us/step - loss: 1.9460 - weighted_acc: 0.1643 - val_loss: 1.9463 - val_weighted_acc: 0.1220
Epoch 2/100
2708/2708 [==============================] - 0s 8us/step - loss: 1.9452 - weighted_acc: 0.1643 - val_loss: 1.9457 - val_weighted_acc: 0.1620
Epoch 3/100
2708/2708 [==============================] - 0s 8us/step - loss: 1.9445 - weighted_acc: 0.1357 - val_loss: 1.9450 - val_weighted_acc: 0.2180
Epoch 4/100
2708/2708 [==============================] - 0s 8us/step - loss: 1.9436 - weighted_acc: 0.2357 - val_loss: 1.9445 - val_weighted_acc: 0.2360
Epoch 5/100
2708/2708 [==============================] - 0s 8us/step - loss: 1.9426 - weighted_acc: 0.3071 - val_loss: 1.9440 - val_weighted_acc: 0.2640
Epoch 6/100
2708/2708 [==============================] - 0s 8us/step - loss: 1.9417 - weighted_acc: 0.3643 - val_loss: 1.9435 - val_weighted_acc: 0.3040
Epoch 7/100
2708/2708 [=========

Epoch 54/100
2708/2708 [==============================] - 0s 8us/step - loss: 1.8758 - weighted_acc: 0.7857 - val_loss: 1.9072 - val_weighted_acc: 0.6200
Epoch 55/100
2708/2708 [==============================] - 0s 8us/step - loss: 1.8705 - weighted_acc: 0.7786 - val_loss: 1.9063 - val_weighted_acc: 0.6160
Epoch 56/100
2708/2708 [==============================] - 0s 8us/step - loss: 1.8713 - weighted_acc: 0.8143 - val_loss: 1.9054 - val_weighted_acc: 0.6160
Epoch 57/100
2708/2708 [==============================] - 0s 8us/step - loss: 1.8707 - weighted_acc: 0.8000 - val_loss: 1.9045 - val_weighted_acc: 0.6180
Epoch 58/100
2708/2708 [==============================] - 0s 8us/step - loss: 1.8669 - weighted_acc: 0.7571 - val_loss: 1.9035 - val_weighted_acc: 0.6200
Epoch 59/100
2708/2708 [==============================] - 0s 8us/step - loss: 1.8628 - weighted_acc: 0.8214 - val_loss: 1.9025 - val_weighted_acc: 0.6220
Epoch 60/100
2708/2708 [==============================] - 0s 8us/step - loss

In [7]:
# Evaluate model
eval_results = model.evaluate([X, A],
                              y,
                              sample_weight=test_mask,
                              batch_size=N)
print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))

2708/2708 [==============================] - 0s 3us/step
Done.
Test loss: 1.8536092042922974
Test accuracy: 0.6829991936683655
